### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 243: 965 m McLane pump filtered on 0.3 um GF-75

In [3]:
# read the CSV into a dataframe using the pandas read_csv function
peaks243_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_DN50.csv")


# also make a dataframe but keep only peptides  >80% ALC
peaks243_80 = peaks243_50.loc[peaks243_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks243_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks243_80))

#look at the dataframe
peaks243_50.head()

# redundant Peaks peptides >50% ALC in dataframe 1718
# redundant Peaks peptides >80% ALC in dataframe 272


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,3,11876,20170531_ETNP_1000m-0.3um-JA14_01.raw,KYVFVK,6,98,6,392.2419,2,42.26,225000.0,782.4691,0.4,NaN,99 97 96 99 100 99,KYVFVK,CID
1,3,8910,20170531_ETNP_1000m-0.3um-JA14_01.raw,M(+15.99)EAYC(+57.02)VK,7,97,7,458.6999,2,32.06,465000.0,915.3830,2.4,Oxidation (M); Carbamidomethylation,94 99 100 99 97 98 98,M(+15.99)EAYC(+57.02)VK,CID
2,3,24439,20170531_ETNP_1000m-0.3um-JA14_01.raw,VTN(+.98)AQ(+.98)LAEVLAR,12,97,12,643.8518,2,79.86,100000000.0,1285.6877,1.0,Deamidation (NQ),96 99 100 100 100 100 99 99 98 98 97 86,VTN(+.98)AQ(+.98)LAEVLAR,CID
3,3,20313,20170531_ETNP_1000m-0.3um-JA14_01.raw,LGSLLDLR(+14.02),8,97,8,450.7797,2,69.26,141000.0,899.5440,0.9,Methylation(KR),97 95 98 99 98 96 98 98,LGSLLDLR(+14.02),CID
4,3,10606,20170531_ETNP_1000m-0.3um-JA14_01.raw,LN(+.98)LAQNWR,8,97,8,508.2628,2,37.93,407000.0,1014.5247,-13.5,Deamidation (NQ),97 98 100 100 96 96 99 95,LN(+.98)LAQNWR,CID


In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks243_50['A'] = peaks243_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks243_50['C'] = peaks243_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks243_50['D'] = peaks243_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks243_50['E'] = peaks243_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks243_50['F'] = peaks243_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks243_50['G'] = peaks243_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks243_50['H'] = peaks243_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks243_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks243_50['I'] = peaks243_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks243_50['K'] = peaks243_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks243_50['I/L'] = peaks243_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks243_50['M'] = peaks243_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks243_50['N'] = peaks243_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks243_50['P'] = peaks243_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks243_50['Q'] = peaks243_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks243_50['R'] = peaks243_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks243_50['S'] = peaks243_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks243_50['T'] = peaks243_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks243_50['V'] = peaks243_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks243_50['W'] = peaks243_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks243_50['Y'] = peaks243_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks243_50['c-carb'] = peaks243_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks243_50['m-oxid'] = peaks243_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks243_50['n-deam'] = peaks243_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks243_50['q-deam'] = peaks243_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks243_50['k-hydr'] = peaks243_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks243_50['p-hydr'] = peaks243_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks243_50['r-meth'] = peaks243_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks243_50['stripped peptide'] = peaks243_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks243_50['stripped length'] = peaks243_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks243_50['ptm-total'] = peaks243_50['c-carb'] + peaks243_50['m-oxid'] + peaks243_50['n-deam'] + peaks243_50['q-deam'] + peaks243_50['k-hydr'] + peaks243_50['p-hydr'] + peaks243_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks243_50['NAAF num.'] = peaks243_50['Area'] / peaks243_50['stripped length']

# write modified dataframe to new csv file
peaks243_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN50.csv")


# check out the results
peaks243_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,3,11876,20170531_ETNP_1000m-0.3um-JA14_01.raw,KYVFVK,6,98,6,392.2419,2,42.26,...,0,0,0,0,0,0,KYVFVK,6,0,37500.0
1,3,8910,20170531_ETNP_1000m-0.3um-JA14_01.raw,M(+15.99)EAYC(+57.02)VK,7,97,7,458.6999,2,32.06,...,1,0,0,0,0,0,MVK,3,2,155000.0
2,3,24439,20170531_ETNP_1000m-0.3um-JA14_01.raw,VTN(+.98)AQ(+.98)LAEVLAR,12,97,12,643.8518,2,79.86,...,0,1,1,0,0,0,VTNLAEVLAR,10,2,10000000.0
3,3,20313,20170531_ETNP_1000m-0.3um-JA14_01.raw,LGSLLDLR(+14.02),8,97,8,450.7797,2,69.26,...,0,0,0,0,0,1,LGSLLDLR,8,1,17625.0
4,3,10606,20170531_ETNP_1000m-0.3um-JA14_01.raw,LN(+.98)LAQNWR,8,97,8,508.2628,2,37.93,...,0,1,0,0,0,0,LNLAQNWR,8,1,50875.0


In [5]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks243_50['A'].sum(),
        'C': peaks243_50['C'].sum(),
        'D': peaks243_50['D'].sum(),
        'E': peaks243_50['E'].sum(),
        'F': peaks243_50['F'].sum(),
        'G': peaks243_50['G'].sum(),
        'H': peaks243_50['H'].sum(),
        'I': peaks243_50['I'].sum(),
        'K': peaks243_50['K'].sum(),
        'I/L': peaks243_50['I/L'].sum(),
        'M': peaks243_50['M'].sum(),
        'N': peaks243_50['N'].sum(),
        'P': peaks243_50['P'].sum(),
        'Q': peaks243_50['Q'].sum(),
        'R': peaks243_50['R'].sum(),
        'S': peaks243_50['S'].sum(),
        'T': peaks243_50['T'].sum(),
        'V': peaks243_50['V'].sum(),
        'W': peaks243_50['W'].sum(),
        'Y': peaks243_50['Y'].sum(),
        'c-carb': peaks243_50['c-carb'].sum(),
        'm-oxid': peaks243_50['m-oxid'].sum(),
        'n-deam': peaks243_50['n-deam'].sum(),
        'q-deam': peaks243_50['q-deam'].sum(),
        'k-hydr': peaks243_50['k-hydr'].sum(),
        'p-hydr': peaks243_50['p-hydr'].sum(),
        'r-meth': peaks243_50['r-meth'].sum(),
        'Total area': peaks243_50['Area'].sum(),
        'Total length': peaks243_50['stripped length'].sum()
       }

totalpeaks243_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks243_50['% C w/ carb'] = totalpeaks243_50['c-carb'] / totalpeaks243_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks243_50['% M w/ oxid'] = totalpeaks243_50['m-oxid'] / totalpeaks243_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks243_50['% N w/ deam'] = totalpeaks243_50['n-deam'] / totalpeaks243_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks243_50['% Q w/ deam'] = totalpeaks243_50['q-deam'] / totalpeaks243_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks243_50['% K w/ hydr'] = totalpeaks243_50['k-hydr'] / totalpeaks243_50['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks243_50['% P w/ hydr'] = totalpeaks243_50['p-hydr'] / totalpeaks243_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks243_50['% R w/ meth'] = totalpeaks243_50['r-meth'] / totalpeaks243_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks243_50['NAAF denom.'] = totalpeaks243_50['Total area'] / totalpeaks243_50['Total length']

# write modified dataframe to new txt file
totalpeaks243_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN50_totals.csv")

totalpeaks243_50.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,986,232,527,683,456,521,446,0,1804,1239,...,3.938148e+09,11550,1.0,0.397866,0.198492,0.034335,0.20898,0.252217,0.429719,340965.21579


In [7]:
# use the calculated NAAF factor (in totalpeaks243 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 340965.21579

# use NAAF >50% ALC to get NAAF
peaks243_50['NAAF factor'] = (peaks243_50['NAAF num.'])/NAAF50

# separate out the dataframe into AAs 
peaks243_AA50 = peaks243_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaks243_AA50['A-NAAF50'] = peaks243_AA50['A'] * peaks243_50['NAAF factor']
peaks243_AA50['C-NAAF50'] = peaks243_AA50['C'] * peaks243_50['NAAF factor']
peaks243_AA50['D-NAAF50'] = peaks243_AA50['D'] * peaks243_50['NAAF factor']
peaks243_AA50['E-NAAF50'] = peaks243_AA50['E'] * peaks243_50['NAAF factor']
peaks243_AA50['F-NAAF50'] = peaks243_AA50['F'] * peaks243_50['NAAF factor']
peaks243_AA50['G-NAAF50'] = peaks243_AA50['G'] * peaks243_50['NAAF factor']
peaks243_AA50['H-NAAF50'] = peaks243_AA50['H'] * peaks243_50['NAAF factor']
peaks243_AA50['K-NAAF50'] = peaks243_AA50['K'] * peaks243_50['NAAF factor']
peaks243_AA50['I/L-NAAF50'] = peaks243_AA50['I/L'] * peaks243_50['NAAF factor']
peaks243_AA50['M-NAAF50'] = peaks243_AA50['M'] * peaks243_50['NAAF factor']
peaks243_AA50['N-NAAF50'] = peaks243_AA50['N'] * peaks243_50['NAAF factor']
peaks243_AA50['P-NAAF50'] = peaks243_AA50['P'] * peaks243_50['NAAF factor']
peaks243_AA50['Q-NAAF50'] = peaks243_AA50['Q'] * peaks243_50['NAAF factor']
peaks243_AA50['R-NAAF50'] = peaks243_AA50['R'] * peaks243_50['NAAF factor']
peaks243_AA50['S-NAAF50'] = peaks243_AA50['S'] * peaks243_50['NAAF factor']
peaks243_AA50['T-NAAF50'] = peaks243_AA50['T'] * peaks243_50['NAAF factor']
peaks243_AA50['V-NAAF50'] = peaks243_AA50['V'] * peaks243_50['NAAF factor']
peaks243_AA50['W-NAAF50'] = peaks243_AA50['W'] * peaks243_50['NAAF factor']
peaks243_AA50['Y-NAAF50'] = peaks243_AA50['Y'] * peaks243_50['NAAF factor']

# write the dataframe to a new csv
peaks243_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN50_AA_NAAF.csv")

peaks243_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,M-NAAF50,N-NAAF50,P-NAAF50,Q-NAAF50,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50
0,KYVFVK,0.109982,0,0,0,0,1,0,0,2,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.219964,0.000000,0.109982
1,MVK,0.454592,1,1,0,1,0,0,0,1,...,0.454592,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.454592,0.000000,0.454592
2,VTNLAEVLAR,29.328505,3,0,0,1,0,0,0,0,...,0.000000,29.328505,0.0,29.328505,29.328505,0.000000,29.328505,58.657010,0.000000,0.000000
3,LGSLLDLR,0.051691,0,0,1,0,0,1,0,0,...,0.000000,0.000000,0.0,0.000000,0.051691,0.051691,0.000000,0.000000,0.000000,0.000000
4,LNLAQNWR,0.149209,1,0,0,0,0,0,0,0,...,0.000000,0.298418,0.0,0.149209,0.149209,0.000000,0.000000,0.000000,0.149209,0.000000


### Same process but for de novo peptide >80 % ALC:

In [8]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks243_80['A'] = peaks243_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks243_80['C'] = peaks243_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks243_80['D'] = peaks243_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks243_80['E'] = peaks243_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks243_80['F'] = peaks243_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks243_80['G'] = peaks243_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks243_80['H'] = peaks243_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks243_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks243_80['I'] = peaks243_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks243_80['K'] = peaks243_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks243_80['I/L'] = peaks243_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks243_80['M'] = peaks243_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks243_80['N'] = peaks243_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks243_80['P'] = peaks243_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks243_80['Q'] = peaks243_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks243_80['R'] = peaks243_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks243_80['S'] = peaks243_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks243_80['T'] = peaks243_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks243_80['V'] = peaks243_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks243_80['W'] = peaks243_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks243_80['Y'] = peaks243_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks243_80['c-carb'] = peaks243_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks243_80['m-oxid'] = peaks243_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks243_80['n-deam'] = peaks243_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks243_80['q-deam'] = peaks243_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks243_80['k-hydr'] = peaks243_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks243_80['p-hydr'] = peaks243_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks243_80['r-meth'] = peaks243_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks243_80['stripped peptide'] = peaks243_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks243_80['stripped length'] = peaks243_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks243_80['ptm-total'] = peaks243_80['c-carb'] + peaks243_80['m-oxid'] + peaks243_80['n-deam'] + peaks243_80['q-deam'] + peaks243_80['k-hydr'] + peaks243_80['p-hydr'] + peaks243_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks243_80['NAAF num.'] = peaks243_80['Area'] / peaks243_80['stripped length']

# write modified dataframe to new csv file
peaks243_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN80.csv")

# check out the results
peaks243_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,3,11876,20170531_ETNP_1000m-0.3um-JA14_01.raw,KYVFVK,6,98,6,392.2419,2,42.26,...,0,0,0,0,0,0,KYVFVK,6,0,37500.0
1,3,8910,20170531_ETNP_1000m-0.3um-JA14_01.raw,M(+15.99)EAYC(+57.02)VK,7,97,7,458.6999,2,32.06,...,1,0,0,0,0,0,MVK,3,2,155000.0
2,3,24439,20170531_ETNP_1000m-0.3um-JA14_01.raw,VTN(+.98)AQ(+.98)LAEVLAR,12,97,12,643.8518,2,79.86,...,0,1,1,0,0,0,VTNLAEVLAR,10,2,10000000.0
3,3,20313,20170531_ETNP_1000m-0.3um-JA14_01.raw,LGSLLDLR(+14.02),8,97,8,450.7797,2,69.26,...,0,0,0,0,0,1,LGSLLDLR,8,1,17625.0
4,3,10606,20170531_ETNP_1000m-0.3um-JA14_01.raw,LN(+.98)LAQNWR,8,97,8,508.2628,2,37.93,...,0,1,0,0,0,0,LNLAQNWR,8,1,50875.0


In [9]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks243_80['A'].sum(),
        'C': peaks243_80['C'].sum(),
        'D': peaks243_80['D'].sum(),
        'E': peaks243_80['E'].sum(),
        'F': peaks243_80['F'].sum(),
        'G': peaks243_80['G'].sum(),
        'H': peaks243_80['H'].sum(),
        'I': peaks243_80['I'].sum(),
        'K': peaks243_80['K'].sum(),
        'I/L': peaks243_80['I/L'].sum(),
        'M': peaks243_80['M'].sum(),
        'N': peaks243_80['N'].sum(),
        'P': peaks243_80['P'].sum(),
        'Q': peaks243_80['Q'].sum(),
        'R': peaks243_80['R'].sum(),
        'S': peaks243_80['S'].sum(),
        'T': peaks243_80['T'].sum(),
        'V': peaks243_80['V'].sum(),
        'W': peaks243_80['W'].sum(),
        'Y': peaks243_80['Y'].sum(),
        'c-carb': peaks243_80['c-carb'].sum(),
        'm-oxid': peaks243_80['m-oxid'].sum(),
        'n-deam': peaks243_80['n-deam'].sum(),
        'q-deam': peaks243_80['q-deam'].sum(),
        'k-hydr': peaks243_80['k-hydr'].sum(),
        'p-hydr': peaks243_80['p-hydr'].sum(),
        'r-meth': peaks243_80['r-meth'].sum(),
        'Total area': peaks243_80['Area'].sum(),
        'Total length': peaks243_80['stripped length'].sum()
       }

totalpeaks243_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks243_80['% C w/ carb'] = totalpeaks243_80['c-carb'] / totalpeaks243_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks243_80['% M w/ oxid'] = totalpeaks243_80['m-oxid'] / totalpeaks243_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks243_80['% N w/ deam'] = totalpeaks243_80['n-deam'] / totalpeaks243_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks243_80['% Q w/ deam'] = totalpeaks243_80['q-deam'] / totalpeaks243_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks243_80['% K w/ hydr'] = totalpeaks243_80['k-hydr'] / totalpeaks243_80['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks243_80['% P w/ hydr'] = totalpeaks243_80['p-hydr'] / totalpeaks243_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks243_80['% R w/ meth'] = totalpeaks243_80['r-meth'] / totalpeaks243_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks243_80['NAAF denom.'] = totalpeaks243_80['Total area'] / totalpeaks243_80['Total length']

# write modified dataframe to new txt file
totalpeaks243_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN80_totals.csv")

totalpeaks243_80.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,193,18,92,131,70,91,24,0,183,295,...,3.229073e+09,2037,1.0,0.571429,0.312,0.079545,0.169399,0.311475,0.276042,1.585210e+06


In [10]:
# use the calculated NAAF factor (in totalpeaks243 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 1.585210e+06

# use NAAF >80% ALC to get NAAF
peaks243_80['NAAF factor'] = (peaks243_80['NAAF num.'])/NAAF80

# separate out the dataframe into AAs 
peaks243_AA80 = peaks243_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy()

# multiply the NAAF80 factor by the AA total to normalize its abundance by peak area and peptide length

peaks243_AA80['A-NAAF80'] = peaks243_AA80['A'] * peaks243_80['NAAF factor']
peaks243_AA80['C-NAAF80'] = peaks243_AA80['C'] * peaks243_80['NAAF factor']
peaks243_AA80['D-NAAF80'] = peaks243_AA80['D'] * peaks243_80['NAAF factor']
peaks243_AA80['E-NAAF80'] = peaks243_AA80['E'] * peaks243_80['NAAF factor']
peaks243_AA80['F-NAAF80'] = peaks243_AA80['F'] * peaks243_80['NAAF factor']
peaks243_AA80['G-NAAF80'] = peaks243_AA80['G'] * peaks243_80['NAAF factor']
peaks243_AA80['H-NAAF80'] = peaks243_AA80['H'] * peaks243_80['NAAF factor']
peaks243_AA80['K-NAAF80'] = peaks243_AA80['K'] * peaks243_80['NAAF factor']
peaks243_AA80['I/L-NAAF80'] = peaks243_AA80['I/L'] * peaks243_80['NAAF factor']
peaks243_AA80['M-NAAF80'] = peaks243_AA80['M'] * peaks243_80['NAAF factor']
peaks243_AA80['N-NAAF80'] = peaks243_AA80['N'] * peaks243_80['NAAF factor']
peaks243_AA80['P-NAAF80'] = peaks243_AA80['P'] * peaks243_80['NAAF factor']
peaks243_AA80['Q-NAAF80'] = peaks243_AA80['Q'] * peaks243_80['NAAF factor']
peaks243_AA80['R-NAAF80'] = peaks243_AA80['R'] * peaks243_80['NAAF factor']
peaks243_AA80['S-NAAF80'] = peaks243_AA80['S'] * peaks243_80['NAAF factor']
peaks243_AA80['T-NAAF80'] = peaks243_AA80['T'] * peaks243_80['NAAF factor']
peaks243_AA80['V-NAAF80'] = peaks243_AA80['V'] * peaks243_80['NAAF factor']
peaks243_AA80['W-NAAF80'] = peaks243_AA80['W'] * peaks243_80['NAAF factor']
peaks243_AA80['Y-NAAF80'] = peaks243_AA80['Y'] * peaks243_80['NAAF factor']

# write the dataframe to a new csv
peaks243_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_DN80_AA_NAAF.csv")

peaks243_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,M-NAAF80,N-NAAF80,P-NAAF80,Q-NAAF80,R-NAAF80,S-NAAF80,T-NAAF80,V-NAAF80,W-NAAF80,Y-NAAF80
0,KYVFVK,0.023656,0,0,0,0,1,0,0,2,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047312,0.000000,0.023656
1,MVK,0.097779,1,1,0,1,0,0,0,1,...,0.097779,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.097779,0.000000,0.097779
2,VTNLAEVLAR,6.308312,3,0,0,1,0,0,0,0,...,0.000000,6.308312,0.0,6.308312,6.308312,0.000000,6.308312,12.616625,0.000000,0.000000
3,LGSLLDLR,0.011118,0,0,1,0,0,1,0,0,...,0.000000,0.000000,0.0,0.000000,0.011118,0.011118,0.000000,0.000000,0.000000,0.000000
4,LNLAQNWR,0.032094,1,0,0,0,0,0,0,0,...,0.000000,0.064187,0.0,0.032094,0.032094,0.000000,0.000000,0.000000,0.032094,0.000000


### Visualizing the results

In [ ]:
print("ALC max: ", peaks['ALC (%)'].max())
print("ALC min: ", peaks['ALC (%)'].min())

In [ ]:
# take only AA totals and transpose for easier bar plotting in matplotlib

peaksaatot = totalpeaks[['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy().T

# take only AA %s and transpose for easier bar plotting in matplotlib

peaksreltot = totalpeaks[['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth']].copy().T

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0


x_labels = ['sample total']

ax = totalpeaks.plot(y=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], kind="bar", title = '965 m suspended')
plt.xticks(rotation=0)
ax.get_legend().remove()
ax.set_xticklabels(x_labels)

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0

my_colors = [(x/10.0, x/20.0, 0.75) for x in range(len(peaksaatot))] # <-- Quick gradient example along the Red/Green dimensions.

ax = peaksaatot.plot(y=['sample total'], kind="bar", color = 'green', title = '965 m suspended')


In [ ]:
# bar plot of relative modifications

ax = totalpeaks.plot(y=['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth'], kind="bar", title = '965 m suspended')
ax.set_xticklabels([])

In [ ]:
# bar plot of relative mods


ax = peaksreltot.plot(y=['sample total'], kind="bar", title = '965 m suspended')
plt.xticks(rotation=45)

In [ ]:
# making evenly spaced bins for the ALC data based on the min and max, called above
bins = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
labels = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']

# use pandas cut function to do the binning itself
peaks['binned'] = pd.cut(peaks['ALC (%)'], bins=bins, labels=labels)

# bar plots of binned PTM data

index = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']
data = {'Total PTMs': [peaks.groupby('binned')['ptm-total'].sum()['50-55'], peaks.groupby('binned')['ptm-total'].sum()['55-60'], peaks.groupby('binned')['ptm-total'].sum()['60-65'], peaks.groupby('binned')['ptm-total'].sum()['65-70'], peaks.groupby('binned')['ptm-total'].sum()['70-75'], peaks.groupby('binned')['ptm-total'].sum()['75-80'], peaks.groupby('binned')['ptm-total'].sum()['80-85'], peaks.groupby('binned')['ptm-total'].sum()['85-90'], peaks.groupby('binned')['ptm-total'].sum()['90-95'], peaks.groupby('binned')['ptm-total'].sum()['95-100']],
        'Cys carb.': [peaks.groupby('binned')['c-carb'].sum()['50-55'], peaks.groupby('binned')['c-carb'].sum()['55-60'], peaks.groupby('binned')['c-carb'].sum()['60-65'], peaks.groupby('binned')['c-carb'].sum()['65-70'], peaks.groupby('binned')['c-carb'].sum()['70-75'], peaks.groupby('binned')['c-carb'].sum()['75-80'], peaks.groupby('binned')['c-carb'].sum()['80-85'], peaks.groupby('binned')['c-carb'].sum()['85-90'], peaks.groupby('binned')['c-carb'].sum()['90-95'], peaks.groupby('binned')['c-carb'].sum()['95-100']],
        'Met oxi.': [peaks.groupby('binned')['m-oxid'].sum()['50-55'], peaks.groupby('binned')['m-oxid'].sum()['55-60'], peaks.groupby('binned')['m-oxid'].sum()['60-65'], peaks.groupby('binned')['m-oxid'].sum()['65-70'], peaks.groupby('binned')['m-oxid'].sum()['70-75'], peaks.groupby('binned')['m-oxid'].sum()['75-80'], peaks.groupby('binned')['m-oxid'].sum()['80-85'], peaks.groupby('binned')['m-oxid'].sum()['85-90'], peaks.groupby('binned')['m-oxid'].sum()['90-95'], peaks.groupby('binned')['m-oxid'].sum()['95-100']],
        'Asp deam.': [peaks.groupby('binned')['n-deam'].sum()['50-55'], peaks.groupby('binned')['n-deam'].sum()['55-60'], peaks.groupby('binned')['n-deam'].sum()['60-65'], peaks.groupby('binned')['n-deam'].sum()['65-70'], peaks.groupby('binned')['n-deam'].sum()['70-75'], peaks.groupby('binned')['n-deam'].sum()['75-80'], peaks.groupby('binned')['n-deam'].sum()['80-85'], peaks.groupby('binned')['n-deam'].sum()['85-90'], peaks.groupby('binned')['n-deam'].sum()['90-95'], peaks.groupby('binned')['n-deam'].sum()['95-100']],
        'Glut deam.': [peaks.groupby('binned')['q-deam'].sum()['50-55'], peaks.groupby('binned')['q-deam'].sum()['55-60'], peaks.groupby('binned')['q-deam'].sum()['60-65'], peaks.groupby('binned')['q-deam'].sum()['65-70'], peaks.groupby('binned')['q-deam'].sum()['70-75'], peaks.groupby('binned')['q-deam'].sum()['75-80'], peaks.groupby('binned')['q-deam'].sum()['80-85'], peaks.groupby('binned')['q-deam'].sum()['85-90'], peaks.groupby('binned')['q-deam'].sum()['90-95'], peaks.groupby('binned')['q-deam'].sum()['95-100']],
        'Lys hydr': [peaks.groupby('binned')['k-hydr'].sum()['50-55'], peaks.groupby('binned')['k-hydr'].sum()['55-60'], peaks.groupby('binned')['k-hydr'].sum()['60-65'], peaks.groupby('binned')['k-hydr'].sum()['65-70'], peaks.groupby('binned')['k-hydr'].sum()['70-75'], peaks.groupby('binned')['k-hydr'].sum()['75-80'], peaks.groupby('binned')['k-hydr'].sum()['80-85'], peaks.groupby('binned')['k-hydr'].sum()['85-90'], peaks.groupby('binned')['k-hydr'].sum()['90-95'], peaks.groupby('binned')['k-hydr'].sum()['95-100']],
        'Pro hydr': [peaks.groupby('binned')['p-hydr'].sum()['50-55'], peaks.groupby('binned')['p-hydr'].sum()['55-60'], peaks.groupby('binned')['p-hydr'].sum()['60-65'], peaks.groupby('binned')['p-hydr'].sum()['65-70'], peaks.groupby('binned')['p-hydr'].sum()['70-75'], peaks.groupby('binned')['p-hydr'].sum()['75-80'], peaks.groupby('binned')['p-hydr'].sum()['80-85'], peaks.groupby('binned')['p-hydr'].sum()['85-90'], peaks.groupby('binned')['p-hydr'].sum()['90-95'], peaks.groupby('binned')['p-hydr'].sum()['95-100']],
        'Lys meth.': [peaks.groupby('binned')['k-meth'].sum()['50-55'], peaks.groupby('binned')['k-meth'].sum()['55-60'], peaks.groupby('binned')['k-meth'].sum()['60-65'], peaks.groupby('binned')['k-meth'].sum()['65-70'], peaks.groupby('binned')['k-meth'].sum()['70-75'], peaks.groupby('binned')['k-meth'].sum()['75-80'], peaks.groupby('binned')['k-meth'].sum()['80-85'], peaks.groupby('binned')['k-meth'].sum()['85-90'], peaks.groupby('binned')['k-meth'].sum()['90-95'], peaks.groupby('binned')['k-meth'].sum()['95-100']],
        'Arg meth.': [peaks.groupby('binned')['r-meth'].sum()['50-55'], peaks.groupby('binned')['r-meth'].sum()['55-60'], peaks.groupby('binned')['r-meth'].sum()['60-65'], peaks.groupby('binned')['r-meth'].sum()['65-70'], peaks.groupby('binned')['r-meth'].sum()['70-75'], peaks.groupby('binned')['r-meth'].sum()['75-80'], peaks.groupby('binned')['r-meth'].sum()['80-85'], peaks.groupby('binned')['r-meth'].sum()['85-90'], peaks.groupby('binned')['r-meth'].sum()['90-95'], peaks.groupby('binned')['r-meth'].sum()['95-100']]
        }

peaksbin = pd.DataFrame(data, columns=['Total PTMs','Cys carb.','Met oxi.','Asp deam.', 'Glut deam.', 'Lys hydr', 'Pro hydr', 'Lys meth.', 'Arg meth.'], index=index)

# write the peaks bin ptm dataframe to a csv:
peaksbin.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_DN50_ptm.csv")

ax1 = peaksbin.plot.bar(y='Total PTMs', rot=45)
ax1.set_title('Total PTMs')

ax2 = peaksbin.plot.bar(y='Cys carb.', rot=45)
ax2.set_title('Cysteine carbamidomethylation')

ax3 = peaksbin.plot.bar(y='Met oxi.', rot=45)
ax3.set_title('Methionine oxidation')

ax4 = peaksbin.plot.bar(y='Asp deam.', rot=45)
ax4.set_title('Asparagine deamidation')

ax5 = peaksbin.plot.bar(y='Glut deam.', rot=45)
ax5.set_title('Glutamine deamidation')

ax6 = peaksbin.plot.bar(y='Lys hydr', rot=45)
ax6.set_title('Lysine hydroxylation')

ax7 = peaksbin.plot.bar(y='Pro hydr', rot=45)
ax7.set_title('Proline hydroxylation')

ax8 = peaksbin.plot.bar(y='Lys meth.', rot=45)
ax8.set_title('Lysine methylation')

ax9 = peaksbin.plot.bar(y='Arg meth.', rot=45)
ax9.set_title('Arginine methylation')


In [ ]:
peaksbin['Total PTMs'] = pd.to_numeric(peaksbin['Total PTMs'])

peaksbin.loc['sums'] = peaksbin.sum(numeric_only=True, axis=0)

peaksbin.loc['rel'] = peaksbin.div(2)['sums']

print(peaksbin)

In [ ]:
# make a new dataframe of relative PTM occurance

# fist pass as numeric
peaks['ptm-total', 'c-carb'] = pd.to_numeric(peaks['ptm-total', 'c-carb'])

data = {'1': [peaks['ptm-total'].sum()], '2': [peaks['c-carb'].sum()]}
#data = {peaks[['ptm-total', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'k-meth', 'r-meth' ]].sum()}

ptmrel = pd.DataFrame(data, columns = ['1', '2'])

ptmrel.head()

### Exporting txt files of stripped peptides at confidence cutoffs:

In [ ]:
# keep only peptide column >50% ALC
pep = peaks[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_DN50_stripped_peptides.txt", header=False, index=False)


# look

print("# of DN peptide >50% ALC", len(pep))
pep.head()

In [ ]:
# keep only peptides  >80% ALC
peaks80 = peaks.loc[peaks['ALC (%)'] >= 80]

# see how many rows and double check
# peaks80.head(-10)

# keep only peptide column 
pep80 = peaks80[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-243-965m-0.3-JA14_DN80_stripped_peptides.txt", header=False, index=False)


print("# of DN peptide >80% ALC", len(pep80))
pep80.head()